# Data cleaning

In [67]:
# all imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

things done before splitting train dataset:
- large extreme energy values are dropped (2 of those, handpicked)
- observations where area/bedroom ratio doesn't make sense are dropped
- some columns are dropped: sticker, price_drop_date, lat, lon, is_promoted
- delete occurences where subtype, energy_value or energy_label are missing (we will need subtype for other energy values later on)

then we split into train validation test sets.

In [68]:
# load datasets
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

# -------- removing invalid outliers --------
# deleteting the 2 large impossible values from the training set for energy_value
train_df = train_df.drop(train_df.nlargest(2, 'energy_value').index)

# deleting impossible area/bedroom ratios
# minimum area per bedroom
min_area_per_bedroom = 6

# area per bedroom ratio
train_df["area_per_bedroom"] = train_df["area"] / train_df["bedrooms"]

# bedroom size is not logical
invalid_outliers = train_df[train_df["area_per_bedroom"] < min_area_per_bedroom]
train_df = train_df.drop(invalid_outliers.index)

# drop the temporary column
train_df = train_df.drop(columns=["area_per_bedroom"])

# delete occurences where subtype, energy_value or energy_label are missing (we will need subtype for other energy values later on)
train_df = train_df.drop(train_df[train_df['subtype'].isna() & (train_df['energy_label'].isna() | train_df['energy_value'].isna())].index)

# remove rows where advertiser is "Your Home" (only applies to train)
train_df = train_df[train_df["advertiser"] != "Your Home"]


# -------- removing not needed columns --------
columns_to_drop = ["sticker", "price_drop_date", "lat", "lon", "is_promoted"]
train_df = train_df.drop(columns=columns_to_drop, errors="ignore")
test_df = test_df.drop(columns=columns_to_drop, errors="ignore")



In [69]:
# split train dataset
target_col = "price"

train_set, val_set, y_train, y_val = train_test_split(
    train_df.drop(columns=[target_col]),  # features only
    train_df[target_col],                 # target
    test_size=0.2, random_state=42
)

- specific advertiser removed (accounts for  alot of missing values)
- for all energy_values above 10000, replace with median of the energy_label
- if energy value is below 0 but label is missing, assign a++ label

In [70]:

# -------- handling outliers --------
# compute median energy_value per energy_label (only from train)
energy_label_mapping = train_set.groupby('energy_label')['energy_value'].median().to_dict()

# replace outliers with the median of their energy_label (only in train)
def replace_outlier(row):
    if row["energy_value"] > 10000:
        return energy_label_mapping.get(row["energy_label"], row["energy_value"])  # use median if available
    return row["energy_value"]

train_set["energy_value"] = train_set.apply(replace_outlier, axis=1)

# do not modify val_set or test_df outliers
# keep extreme values in validation to properly test generalization

# -------- energy_value below 0 --------
# only in train
train_set.loc[(train_set['energy_value'] < 0) & (train_set['energy_label'].isna()), 'energy_label'] = 'a++'

most stats used for mapping are defined here: 
- energy stats: min max median of energy_value per label
- median_area_by_bedrooms: median of area per bedrooms
- median_by_subtype: median of energy_values based on subtype

In [ ]:
# -------- most stats --------
# define energy label hierarchy (higher efficiency first)
label_hierarchy = ["a++", "a+", "a-", "a", "b+", "b-", "b", "c+", "c-", "c", "d+", "d-", "d", "e+", "e-", "e", "f", "g"]

# compute min, max, and median energy_value per label (only from train)
energy_stats = train_set.groupby("energy_label")['energy_value'].agg(['min', 'max', 'median']).dropna()
energy_stats = energy_stats.reindex(label_hierarchy)  # apply hierarchy sorting


# function to find the best label based on closest median energy_value
def impute_label(energy_value):
    if np.isnan(energy_value):
        return np.nan  # skip missing energy_value
    
    # find labels where the energy_value falls within the range
    valid_labels = energy_stats[(energy_stats['min'] <= energy_value) & (energy_value <= energy_stats['max'])]
    
    if valid_labels.empty:
        return np.nan  # no matching label range found
    
    # select the label with the closest median energy_value
    closest_labels = valid_labels.iloc[(valid_labels['median'] - energy_value).abs().argsort()]
    
    # prioritize the highest efficiency label if multiple labels have the same median
    best_label = closest_labels.index[0]
    return best_label

# compute median area per bedroom count (only from train)
median_area_by_bedrooms = train_set.groupby("bedrooms")["area"].median()

# compute median energy_value for each subtype from train_set
median_by_subtype = train_set.groupby('subtype')['energy_value'].median()

# Define the mapping for grouping energy labels
label_grouping = {
    "a++": "a", "a+": "a", "a-": "a",
    "b+": "b", "b-": "b", "b": "b",
    "c+": "c", "c-": "c", "c": "c",
    "d+": "d", "d-": "d", "d": "d",
    "e+": "e", "e-": "e", "e": "e",
    "f": "f", "g": "g"}

the code does:
- if area is missing, fill it with median of the number of bedrooms for the number of bedrooms the listing has
- if energy value is missing and energy label isnt, use the median of the energy label to fill the energy value
- if energy label is missing and energy value isnt, impute the closest label to the value
- if new building is 1 and it still hasn't been filled so far, assign class a (newer buildings more efficient by law)
- if new building is 0 and it still hasnt been filled, map the value based on subtype and then impute closest label to the value
- groups the granular categories into the 7 main categories

In [ ]:

for df in [train_set, val_set, test_df]:
    # fill missing area values
    df["area"] = df["area"].fillna(df["bedrooms"].map(median_area_by_bedrooms))


    # fill missing energy_value using the median of its corresponding energy_label if it exists
    df.loc[df['energy_value'].isnull() & df['energy_label'].notnull(), 'energy_value'] = df['energy_label'].map(energy_label_mapping)

    #!! yes/no?
    df['subtype'] = df['subtype'].fillna('Unknown')
    df['advertiser'] = df['advertiser'].fillna('Unknown')

    # fill missing energy_label based on energy_value
    df.loc[df['energy_label'].isna(), 'energy_label'] = df['energy_value'].apply(impute_label)

    # assign energy_label = 'a' and fill energy_value for new_building == 1
    df.loc[(df['new_building'] == 1) & df['energy_label'].isna(), 'energy_label'] = 'a'
    df.loc[(df['new_building'] == 1) & df['energy_value'].isna(), 'energy_value'] = energy_label_mapping['a']

    # impute missing values in each dataset where new_building = 0
    mask = (df['new_building'] == 0) & (df['energy_value'].isna())
    df.loc[mask, 'energy_value'] = df.loc[mask, 'subtype'].map(median_by_subtype)
    df.loc[(df['new_building'] == 0) & df['energy_label'].isna(), 'energy_label'] = df['energy_value'].apply(impute_label)

    # Group the energy labels according to the label_grouping dictionary
    df['energy_label'] = df['energy_label'].map(label_grouping).fillna(df['energy_label'])

In [75]:
# save cleaned datasets
train_set.to_csv("./data/cleaned_train.csv", index=False)
val_set.to_csv("./data/cleaned_val.csv", index=False)
test_df.to_csv("./data/cleaned_test.csv", index=False)
y_train.to_csv("./data/y_train.csv", index=False)
y_val.to_csv("./data/y_val.csv", index=False)

remaining issues:
- woonboot subtype median is nan (`median_by_subtype['Woonboot']`)
- when bedroom number is 14, the median area is also nan (`median_area_by_bedrooms[14]`)

In [74]:
train_set[train_set.isna().any(axis=1)]

,id,is_appartment,area,added_time,bedrooms,new_building,postcode,advertiser,foto_amount,subtype,energy_value,energy_label,province
18071,tr18071,False,NaN,107248513,14.0,0,6180,Immo STILLAVATI,2.0,Woning,743.0,g,Henegouwen


# The code from above, just condensed in 1 cell for easy executing.

In [ ]:
# all imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# load datasets
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

# -------- removing invalid outliers --------
# deleteting the 2 large impossible values from the training set for energy_value
train_df = train_df.drop(train_df.nlargest(2, 'energy_value').index)

# deleting impossible area/bedroom ratios
# minimum area per bedroom
min_area_per_bedroom = 6

# area per bedroom ratio
train_df["area_per_bedroom"] = train_df["area"] / train_df["bedrooms"]

# bedroom size is not logical
invalid_outliers = train_df[train_df["area_per_bedroom"] < min_area_per_bedroom]
train_df = train_df.drop(invalid_outliers.index)

# drop the temporary column
train_df = train_df.drop(columns=["area_per_bedroom"])

# delete occurences where subtype, energy_value or energy_label are missing (we will need subtype for other energy values later on)
train_df = train_df.drop(train_df[train_df['subtype'].isna() & (train_df['energy_label'].isna() | train_df['energy_value'].isna())].index)

# remove rows where advertiser is "Your Home" (only applies to train)
train_df = train_df[train_df["advertiser"] != "Your Home"]


# -------- removing not needed columns --------
columns_to_drop = ["sticker", "price_drop_date", "lat", "lon", "is_promoted"]
train_df = train_df.drop(columns=columns_to_drop, errors="ignore")
test_df = test_df.drop(columns=columns_to_drop, errors="ignore")

# split train dataset
target_col = "price"

train_set, val_set, y_train, y_val = train_test_split(
    train_df.drop(columns=[target_col]),  # features only
    train_df[target_col],                 # target
    test_size=0.2, random_state=42
)


# -------- handling outliers --------
# compute median energy_value per energy_label (only from train)
energy_label_mapping = train_set.groupby('energy_label')['energy_value'].median().to_dict()

# replace outliers with the median of their energy_label (only in train)
def replace_outlier(row):
    if row["energy_value"] > 10000:
        return energy_label_mapping.get(row["energy_label"], row["energy_value"])  # use median if available
    return row["energy_value"]

train_set["energy_value"] = train_set.apply(replace_outlier, axis=1)

# do not modify val_set or test_df outliers
# keep extreme values in validation to properly test generalization

# -------- energy_value below 0 --------
# only in train
train_set.loc[(train_set['energy_value'] < 0) & (train_set['energy_label'].isna()), 'energy_label'] = 'a++'

# -------- most stats --------
# define energy label hierarchy (higher efficiency first)
label_hierarchy = ["a++", "a+", "a-", "a", "b+", "b-", "b", "c+", "c-", "c", "d+", "d-", "d", "e+", "e-", "e", "f", "g"]

# compute min, max, and median energy_value per label (only from train)
energy_stats = train_set.groupby("energy_label")['energy_value'].agg(['min', 'max', 'median']).dropna()
energy_stats = energy_stats.reindex(label_hierarchy)  # apply hierarchy sorting


# function to find the best label based on closest median energy_value
def impute_label(energy_value):
    if np.isnan(energy_value):
        return np.nan  # skip missing energy_value
    
    # find labels where the energy_value falls within the range
    valid_labels = energy_stats[(energy_stats['min'] <= energy_value) & (energy_value <= energy_stats['max'])]
    
    if valid_labels.empty:
        return np.nan  # no matching label range found
    
    # select the label with the closest median energy_value
    closest_labels = valid_labels.iloc[(valid_labels['median'] - energy_value).abs().argsort()]
    
    # prioritize the highest efficiency label if multiple labels have the same median
    best_label = closest_labels.index[0]
    return best_label

# compute median area per bedroom count (only from train)
median_area_by_bedrooms = train_set.groupby("bedrooms")["area"].median()

# compute median energy_value for each subtype from train_set
median_by_subtype = train_set.groupby('subtype')['energy_value'].median()

# Define the mapping for grouping energy labels
label_grouping = {
    "a++": "a", "a+": "a", "a-": "a",
    "b+": "b", "b-": "b", "b": "b",
    "c+": "c", "c-": "c", "c": "c",
    "d+": "d", "d-": "d", "d": "d",
    "e+": "e", "e-": "e", "e": "e",
    "f": "f", "g": "g"}

for df in [train_set, val_set, test_df]:
    # fill missing area values
    df["area"] = df["area"].fillna(df["bedrooms"].map(median_area_by_bedrooms))


    # fill missing energy_value using the median of its corresponding energy_label if it exists
    df.loc[df['energy_value'].isnull() & df['energy_label'].notnull(), 'energy_value'] = df['energy_label'].map(energy_label_mapping)

    #!! yes/no?
    df['subtype'] = df['subtype'].fillna('Unknown')
    df['advertiser'] = df['advertiser'].fillna('Unknown')

    # fill missing energy_label based on energy_value
    df.loc[df['energy_label'].isna(), 'energy_label'] = df['energy_value'].apply(impute_label)

    # assign energy_label = 'a' and fill energy_value for new_building == 1
    df.loc[(df['new_building'] == 1) & df['energy_label'].isna(), 'energy_label'] = 'a'
    df.loc[(df['new_building'] == 1) & df['energy_value'].isna(), 'energy_value'] = energy_label_mapping['a']

    # impute missing values in each dataset where new_building = 0
    mask = (df['new_building'] == 0) & (df['energy_value'].isna())
    df.loc[mask, 'energy_value'] = df.loc[mask, 'subtype'].map(median_by_subtype)
    df.loc[(df['new_building'] == 0) & df['energy_label'].isna(), 'energy_label'] = df['energy_value'].apply(impute_label)

    # Group the energy labels according to the label_grouping dictionary
    df['energy_label'] = df['energy_label'].map(label_grouping).fillna(df['energy_label'])

# save cleaned datasets
train_set.to_csv("./data/cleaned_train.csv", index=False)
val_set.to_csv("./data/cleaned_val.csv", index=False)
test_df.to_csv("./data/cleaned_test.csv", index=False)
y_train.to_csv("./data/y_train.csv", index=False)
y_val.to_csv("./data/y_val.csv", index=False)